In [82]:
import pandas as pd
import math
import numpy as np


In [83]:
#Inicia os constantes
sizeDF = 8716

#Number os words Documents
numDocWords = 3677943

#Average length documents
avgDocs = numDocWords / float(sizeDF)

In [84]:
#Função para carregar o csv, preencher os espaços NaN e formatar o contéudo que ira ser utilizado
def getCSVFormattedAsDF():
    df = pd.read_csv('estadao_noticias_eleicao.csv')
    df = df.fillna('')
    df["data"] = df["titulo"] + ' ' +  df["subTitulo"] + ' ' + df["conteudo"]

    return df

#Função para carregar o csv do gabarito que será utilizado no experimento
def getGabaritoDF():
    df = pd.read_csv('gabarito.csv')
    return df


In [85]:
"""
 return as informações de tf que serão utilizado durante todo o experimento.
   {"idNoticia1": {"word1": num_ocorrencias}, {"word2": num_ocorrencias}, 
    "idNoticia2": {"word1": num_ocorrencias}, {"word3": num_ocorrencias} ... }
"""
def get_tf():
    tf = {}
    df = getCSVFormattedAsDF()
    sizeDF = len(df["idNoticia"])
    
    for idx in range(sizeDF):
        content = df["data"][idx]
        words = content.split(" ")
        tf[df["idNoticia"][idx]] = {}
        for word in words:
            word = word.lower()
            try:
                tf[df["idNoticia"][idx]][word] = tf[df["idNoticia"][idx]][word] + 1
            except KeyError:
                tf[df["idNoticia"][idx]][word] = 1
            
    return tf

In [86]:
"""
 return o idf de cada palavra
"""
def get_idf(tf):
    tfGeneral = {}
    idf = {}
    for new in tf:
        for word in tf[new]:
            try:
                tfGeneral[word] = tfGeneral[word] + 1
            except KeyError:
                 tfGeneral[word] = 1
    for word in tfGeneral:
        idf[word] = math.log(float(sizeDF)/(tfGeneral[word])) 
    return idf

In [87]:
# Retorna o valor dos top5 documentos pela busca binaria, de acordo com a query pesquisada 
def top5Binario(query, tf):
    doc = {}
    for new in tf:
        doc[new] = 0
        sum = 0
        checkIfConjuntive = True
        for palavra in query:
            if (palavra in tf[new]):
                sum += doc[new] + 1
            else:
                checkIfConjuntive = False
        if (checkIfConjuntive):
            doc[new] = True
    sortedDoc = sorted(doc, key=doc.get, reverse=True)
    
    return sortedDoc[0:5]
 
# Retorna o valor dos top5 documentos pelo tf, de acordo com a query pesquisada 
def top5TF(query, tf):
    doc = {}
    for new in tf:
        doc[new] = 0
        sum = 0
        checkIfConjuntive = True
        for palavra in query:
            if (palavra in tf[new]):
                sum += tf[new][palavra]
            else:
                checkIfConjuntive = False
        if (checkIfConjuntive):
            doc[new] += sum
    sortedDoc = sorted(doc, key=doc.get, reverse=True)

    return sortedDoc[0:5]

# Retorna o valor dos top5 documentos pelo idf, de acordo com a query pesquisada 
def top5TFIDF(query, tf, idf):
    doc = {}
    for new in tf:
        doc[new] = 0
        sum = 0
        checkIfConjuntive = True
        for palavra in query:
            if (palavra in tf[new]):
                sum += tf[new][palavra] * idf[palavra]
            else:
                checkIfConjuntive = False
        if (checkIfConjuntive):
            doc[new] += sum
    sortedDoc = sorted(doc, key=doc.get, reverse=True)

    return sortedDoc[0:5]

# Retorna o valor dos top5 documentos pelo idf, de acordo com a query pesquisada 
def top5bm25(query, tf, idf):
    doc = {}
    k = 1.5
    b = 0.75
    df = getCSVFormattedAsDF()
    
    for new in tf:
        doc[new] = 0
        sum = 0
        checkIfConjuntive = True
        content = df["data"][new-1]
        words = content.split(" ")
        numberDocsNew = len(words)
        for palavra in query:
            if (palavra in tf[new]):
                sum += ((tf[new][palavra]*(k+1.0)) / (tf[new][palavra] + k*0.25 + k*b*(numberDocsNew/avgDocs))) * idf[palavra] 
            else:
                checkIfConjuntive = False
        if(checkIfConjuntive):
            doc[new] += sum
       
    sortedDoc = sorted(doc, key=doc.get, reverse=True)

    return sortedDoc[0:5]

In [88]:
# Funções comparativas do experimento
def apk(actual, predicted, k=10):
    if (len(predicted)>k):
        predicted = predicted[:k]
        
    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if (not actual):
        return 0.0

    return (score / min(len(actual), k))

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [89]:
#Funções para montar as respostas em formato de dicionarios de arrays oriundas do gabarito do experimento
def getGabFormatted(key, idx, gabaritoDF):
    formattedGab = gabaritoDF[key][idx].replace(",", "")
    formattedGab = formattedGab.replace("[", "")
    formattedGab = formattedGab.replace("]", "")
    return list(map(int, formattedGab.split(' ')))

def getGabResult(): 
    gabaritoDF = getGabaritoDF()
    gabResult = {}
    gabResult["tf"] = []
    gabResult["binario"] = []
    gabResult["tfidf"] = []
    gabResult["bm25"] = []

    for idx in range(len(gabaritoDF)):
        gabResult["tf"].append(getGabFormatted("tf", idx, gabaritoDF))
        gabResult["binario"].append(getGabFormatted("busca_binaria",idx, gabaritoDF))
        gabResult["tfidf"].append(getGabFormatted("tfidf",idx, gabaritoDF))
        gabResult["bm25"].append(getGabFormatted("bm25",idx, gabaritoDF))
    return gabResult

In [90]:
#Funções para montar as respostas em formato de dicionarios de arrays oriundas das minhas respostas obtidas no experimento
def getMyResultsFormatted(queries, tf, idf):
    myResult = {}
    myResult["tf"] = []
    myResult["binario"] = []
    myResult["tfidf"] = []
    myResult["bm25"] = []

    for query in queries:
        myResult["binario"].append(top5Binario(query, tf)) 
        myResult["tf"].append(top5TF(query, tf))
        myResult["tfidf"].append(top5TFIDF(query, tf, idf))
        myResult["bm25"].append(top5bm25(query, tf, idf))
    return myResult

In [91]:
#Armezana em memoria o tf
tf = get_tf()
#Armezana em memoria o tf
idf = get_idf(tf)
#Queries que foram utilizadas no experimento
queries = [
    ["segundo", "turno"],
    ["lava", "jato"],
    ["projeto", "de", "lei"],
    ["compra", "de", "voto"],
    ["ministério", "público"]
]

In [92]:
myResult = getMyResultsFormatted(queries, tf, idf)
gabResult = getGabResult()

In [93]:
print(mapk(gabResult["tf"], myResult["tf"], k=5))
print(mapk(gabResult["binario"], myResult["binario"], k=5))
print(mapk(gabResult["tfidf"], myResult["tfidf"], k=5))
print(mapk(gabResult["bm25"], myResult["bm25"], k=5))


0.712
0.12
0.490666666667
0.168666666667


In [94]:
#Explicações:
"""
    1) Método de busca binaria
        Basicamente a ideia deste método é basicamente analisar em todos os documentos existentes a ocorrência ou 
        não da busca feita em cada documento. Não importanto se a palavra aparece um ou mais vezes. A única coisa 
        que interessa é se aparece ou não. 
    2) Método TF:
        O método TF também olha em cada documento se há ocorrência ou não da busca feita. A grande vantagem deste
        método em relação ao de busca binária, é que ele olha também a frequência. Ou seja, a quantidade de vezes
        que as palavras da busca apareceram no documento. Isso é interessante pois dá uma precisão maior na minha 
        busca em documentos especificos.
        
    3) Método TF-IDF:
        Contudo, quando estamos procurando no todo, o resultado da frequencia da busca pode atrapalhar, já que em
        um documento a busca pode aparecer várias vezes e em outro simplesmente não aparecer, gerando uma imprecisão
        da busca no todo. Para isto é utilizando o idf no método, para equilibrar a frequência da palavra no 
        todo.
    4) Método bm25:
        A ideia do BM25 é rankear a pesquisa baseado nos parametros de busca que aparecem em cada documento,
        não importanto a inter-relação entre a pesquisa e o documento. Para mais informações, acredito que
        esse link possa explicar bem melhor do que eu: https://en.wikipedia.org/wiki/Okapi_BM25
    
    Como pode-se notar nos resultados obtidos acima, nota-se que o método mais acertivo em relação ao gabarito foi 
    método TF. Isso pode ter ocorrido, pois há poucas variações do algoritmo deste método em questão¹. Em segundo,
    e em terceiro, temos o tfidf e o bm25. Acredito que isso possa ter acontecido pela diferença no uso do 
    algoritmo em ambos os casos. No bm25 foi utilizado o okapi BM25 e no tf-idf foi utilizado um diferente do 
    apresentado no slide(Porém, correto). Outro ponto é que o bm25 tem os parametros k e b, que acabam por ajudar
    na variação de score encontrado pelo método para a pesquisa. Por último temos a busca binária, a diferença
    se dá pois temos que esse método acabam gerando varios scores iguais, e por isso, na hora do slice acaba-se 
    por haver essa diferença do gabarito e da resposta encontrada.
    
    ¹: Com o uso do nltk apresentou-se resultados bem mais parecidos com o do gabarito
    ²: Encarei a comporação do google com nossas funções como opcional, já que não estava explicitada no documento
    (Ao menos, entendi assim. Peço desculpas caso não era opcional)
"""

'\n    1) Método de busca binaria\n        Basicamente a ideia deste método é basicamente analisar em todos os documentos existentes a ocorrência ou \n        não da busca feita em cada documento. Não importanto se a palavra aparece um ou mais vezes. A única coisa \n        que interessa é se aparece ou não. \n    2) Método TF:\n        O método TF também olha em cada documento se há ocorrência ou não da busca feita. A grande vantagem deste\n        método em relação ao de busca binária, é que ele olha também a frequência. Ou seja, a quantidade de vezes\n        que as palavras da busca apareceram no documento. Isso é interessante pois dá uma precisão maior na minha \n        busca em documentos especificos.\n        \n    3) Método TF-IDF:\n        Contudo, quando estamos procurando no todo, o resultado da frequencia da busca pode atrapalhar, já que em\n        um documento a busca pode aparecer várias vezes e em outro simplesmente não aparecer, gerando uma imprecisão\n        da busc